In [1]:
import numpy as np
import math
import time
import scipy
from math import pi, exp, factorial, sin, cos
from matplotlib import pyplot as plt

In [2]:
def QR_Householder(A): #UPGRADE: Let the algorithm accept non-square matrices?
    n = np.shape(A)[0]
    R = np.copy(A)
    Q = np.eye(n)
    V = np.zeros((n,n))
    for k in range(0,n):
        x = R[k:,k]
        e1 = np.zeros(np.size(x))
        e1[0] = 1
        v = np.sign(x[0])*np.linalg.norm(x,2)*e1 + x
        v = v/np.linalg.norm(v,2)
        V[0:(n-k),k] = v
        F = np.eye(n-k,n-k) - 2*np.outer(v,v)
        R[k:,k:] =  F@R[k:,k:]
        Q[k:,:] = F@np.copy(Q[k:,:]) #Modification to get Q.
    return R, V, np.matrix.getH(Q)
        

In [3]:
def QR_GS_gen(A): #This uses Gram-Schmitz, but can accept non-square matrix inputs.
    n = np.shape(A)
    Q = np.zeros((n[0],n[1]))
    R = np.zeros((n[1],n[1]))
    for j in range(0,n[1]):
        S = A[:,j]
        for i in range(0,j):
            R[i,j] = np.dot(np.matrix.getH(Q[:,i]),A[:,j])
            S = S - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(S)
        Q[:,j] = S/R[j,j]
    return Q, R

In [4]:
def QREig(A, maxiter = 2000, tol = 1e-7, printiter = False):
    n = np.shape(A)[0]
    eig = np.empty(n, dtype = "cfloat")
    ev = np.eye(n)
    if n == 1:
        return A[0,0]
    if n == 2:
        alpha = np.matrix.trace(A[0:2,0:2])
        beta = np.linalg.det(A[0:2,0:2])
        eig[0] = 0.5*(alpha+np.sqrt(alpha**2 - 4*beta + 0j))
        eig[1] = 0.5*(alpha-np.sqrt(alpha**2 - 4*beta + 0j))
        v2_1 = -(A[0,0] - eig1)/A[0,1]
        v2_2 = -(A[0,0] - eig2)/A[0,1]
        return eig, np.matrix([[1,1],[v2_1,v2_2]])
    if n == 3:
        for j in range(1,50):
            AO = A
            R,u,Q = QR_Householder(A) #QR algorithm goes here.
            ev = np.matmul(ev,Q)
            A = np.matmul(R,Q)
    for j in range(1,maxiter):
        AO = A
        R,u,Q = QR_Householder(A) #QR algorithm goes here.
        ev = np.matmul(ev,Q)
        A = np.matmul(R,Q)
        if np.linalg.norm(np.tril(A,-2)) < tol:
            i = 0
            while i < n-1:
                if abs(A[i+1,i]) < tol**0.5:
                    eig[i] = A[i,i]
                    i += 1
                else:
                    alpha = np.matrix.trace(A[i:(i+2),i:(i+2)])
                    beta = np.linalg.det(A[i:(i+2),i:(i+2)])
                    im = np.sqrt(alpha**2 - 4*beta + 0j)
                    eig[i] = 0.5*(alpha+im)
                    eig[i+1] = 0.5*(alpha-im)
                    i += 2
            if i >= n:
                if printiter == True:
                    print("Iterations used =",j)
                return eig, ev
            else:
                eig[n-1] = A[n-1,n-1]
                if printiter == True:
                    print("Iterations used =",j)
                return eig, ev

In [5]:
def SVD(A, maxiter = 2000, tol = 1e-8, printiter = False):
    r = np.shape(A)[0]
    c = np.shape(A)[1]
    GS = False
    if r < c:
        V = np.zeros((c,c))
        #Ueigs = np.linalg.eig(A@np.matrix.getH(A)) #Using Numpy's eigenvalue solver.
        Ueigs = QREig(A@np.matrix.getH(A))
        U = Ueigs[1]
        D = np.hstack((np.diag(Ueigs[0]),np.zeros((r,c-r))))
        for j in range(0,r):
            if D[j,j] < tol: #The eigenvalues should not be negative.
                D[j,j] = 0
                V[:,j] = np.random.random(c)
                GS = True
            else:
                D[j,j] = np.sqrt(D[j,j])
                V[:,j] = np.matmul(np.matrix.getH(A), U[:,j])/D[j,j]
        if GS == True:
            V = np.linalg.qr(V)[0]
    else:
        U = np.zeros((r,r))
        #Veigs = np.linalg.eig(np.matrix.getH(A)@A) #Using Numpy's eigenvalue solver.
        Veigs = QREig(np.matrix.getH(A)@A)
        V = Veigs[1]
        D = np.vstack((np.diag(Veigs[0]),np.zeros((r-c,c))))
        for j in range(0,c):
            if D[j,j] < tol: #The eigenvalues should not be negative.
                D[j,j] = 0
                U[:,j] = np.random.random(r)
                GS = True
            else:
                D[j,j] = np.sqrt(D[j,j])
                U[:,j] = np.matmul(A, V[:,j])/D[j,j]
        if GS == True:
            U = np.linalg.qr(U)[0] #We can also use our own Gram-Schmidt algorithm.
                                   #There may be sign ambiguity errors.
                                   #Our eigenvalue solver may not be properly constructed to deal with this scenario.
    
    return U,D,V

In [8]:
A = np.array([[13,2,3],[8,2,7],[1,4,2],[5,9,6]])
print(A)
U,D,V = SVD(A)
print(U)
print(D)
print(V)
print(np.real(U@D@np.matrix.getH(V)))

[[13  2  3]
 [ 8  2  7]
 [ 1  4  2]
 [ 5  9  6]]
[[ 0.63982798 -0.59116779 -0.52314173  0.        ]
 [ 0.53283156 -0.11389604  0.87558851  0.        ]
 [ 0.17002634  0.38026986 -0.15447242  0.        ]
 [ 0.52999878  0.72010302 -0.17616423  0.        ]]
[[19.48482429+0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  8.21760161+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  3.29733252+0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j]]
[[ 0.79019149 -0.55876327 -0.25175587]
 [ 0.39741658  0.77989077 -0.48356018]
 [ 0.46653775  0.28205319  0.83832483]]
[[13.  2.  3.]
 [ 8.  2.  7.]
 [ 1.  4.  2.]
 [ 5.  9.  6.]]


c:\users\dionh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:34: ComplexWarning: Casting complex values to real discards the imaginary part
